In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/udemy_ml

/content/drive/MyDrive/Colab Notebooks/udemy_ml


In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('wiki_movies.csv')
print(df.head(1))
print(df.keys())
print(df.shape)

   Release Year  ...                                               Plot
0          1901  ...  A bartender is working at a saloon, serving dr...

[1 rows x 8 columns]
Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot'],
      dtype='object')


In [5]:
final_df = df[df['Release Year'] > 2012]
final_df = final_df.drop('Wiki Page', axis=1)
print(final_df.shape)
print(final_df.head(3)['Title'])
print(final_df[final_df['Title'] == '16 Blocks']['Plot'])

(4137, 7)
16409                   2 Guns
16410    12 Rounds 2: Reloaded
16411         12 Years a Slave
Name: Title, dtype: object
Series([], Name: Plot, dtype: object)


# Cleaning

In [6]:
import nltk
import string
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
def clean_sentences(text):
  text = text.lower()
  words = nltk.word_tokenize(text)
  new_sent = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
  new_sent = [word for word in new_sent if word not in string.punctuation]
  new_sent = ' '.join(new_sent)

  new_sent = new_sent.replace("n't",'not')
  new_sent = new_sent.replace("'m",'am')
  new_sent = new_sent.replace("'s",'is')
  new_sent = new_sent.replace("'re",'are')
  new_sent = new_sent.replace("'ll",'will')
  new_sent = new_sent.replace("'d",'would')
  new_sent = new_sent.replace("'ve",'have')
  
  return new_sent

final_df['new_plot'] = final_df['Plot'].apply(clean_sentences)

In [8]:
final_df = final_df.reset_index(drop=True)
final_df.head(2)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Plot,new_plot
0,2013,2 Guns,American,Baltasar Kormákur,"Denzel Washington, Mark Wahlberg, Paula Patton...",action comedy,"Criminals Robert ""Bobby"" Beans (Denzel Washing...",criminal robert `` bobby '' bean denzel washin...
1,2013,12 Rounds 2: Reloaded,American,Roel Reiné,"Randy Orton, Brian Markinson, Venus Terzo",action,Paramedic Nick Malloy and his wife Sarah are g...,paramedic nick malloy wife sarah going home mo...


# TF-IDF

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_movies = tfidf.fit_transform(final_df['new_plot']) 

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

df_to_sim = final_df[['Title', 'new_plot']]
similarity = cosine_similarity(tfidf_movies, tfidf_movies)
indices = pd.Series(final_df.index)

In [19]:
def recommendation(title, cosine_sim=similarity):
  try:
    index = final_df[final_df['Title'] == title].index[0]
    scores = pd.Series(similarity[index]).sort_values(ascending= False)[1:10]
    top_movies = [final_df.iloc[i] for i in scores.index]
    return pd.DataFrame(top_movies)
  except:
    return "No match found"

In [20]:
print(recommendation('Dracula: The Dark Prince')['Title'])

2076                       Race 2
464                Dracula Untold
3854            The Very Best Day
455                 The Equalizer
242               Rapture-Palooza
1112           Summer in February
795          Beauty and the Beast
322           Black Water Vampire
107     Fright Night 2: New Blood
Name: Title, dtype: object
